In [1]:
# General Imports
import os
import json
import aiometer
import asyncio
import time
import anyio
import httpx
import logging
import requests
import functools
import math
from dotenv import load_dotenv
import base64


# Skip helper library Import
import skip

# Crypto/Cosmos Imports
from bip_utils import Bip39SeedGenerator, Bip44, Bip44Coins
from cosmpy.aerial.client import LedgerClient, NetworkConfig
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

from cosmpy.protos.cosmwasm.wasm.v1.query_pb2 import (
    QuerySmartContractStateRequest,
    QuerySmartContractStateResponse)

# Local imports
from mempool import check_for_swap_txs_in_mempool
from update_contracts import update_reserves, update_fees, batch_update_fees, batch_update_reserves, generate_three_pool_cyclic_routes
from swaps import Transaction, Swap, JunoSwap
from calculate import calculate_optimal_amount_in, get_profit_from_route
from create_tx import create_tx
from messages import create_route_msgs
from route import get_route_object

from query_contracts import terraswap_factory

load_dotenv('terra.env')

# All global variables to be used throughout the program

# Path to contracts file
CONTRACTS_FILE = os.environ.get("CONTRACTS_FILE")

# Mnenomic to generate private key
# Replace with your own mnemonic
MNEMONIC = os.environ.get("MNEMONIC")

# RPC URL to stream mempool and query contract state from
RPC_URL = os.environ.get("RPC_URL")

# Rest URL to use to get a network client from cosmpy 
REST_URL = os.environ.get("REST_URL")

# Chain ID of network, to be used for network client
# You can find chain IDs at https://github.com/cosmos/chain-registry
CHAIN_ID = os.environ.get("CHAIN_ID")

# Fee asset denom to for network, to be used for network client
FEE_DENOM = os.environ.get("FEE_DENOM")

# Total gas limit for transaction
# This can be optimized in the future to be dynamic
# Based on the number of messages in the transaction
GAS_LIMIT = int(os.environ.get("GAS_LIMIT"))

# Price per gast unit to calculate total fee paid for has
GAS_PRICE = float(os.environ.get("GAS_PRICE"))

# Gas fee to be paid for transaction
# Calculated by multiplying gas limit and gas price
GAS_FEE = int(GAS_LIMIT * GAS_PRICE)

# Addrex prefix for the network, to be used to generate wallet object
ADDRESS_PREFIX = os.environ.get("ADDRESS_PREFIX")

# RPC URL to send skip bundle to
# Can find the respective url at: 
# https://skip-protocol.notion.site/Skip-Configurations-By-Chain-a6076cfa743f4ab38194096403e62f3c
SKIP_RPC_URL = os.environ.get("SKIP_RPC_URL")

# Address to send bid payment to for skip's blockspace auction
AUCTION_HOUSE_ADDRESS = os.environ.get("AUCTION_HOUSE_ADDRESS")

# The percentage of the arb profit
# To be used as the bid to the Skip Auction
# Note: There will probably be an equilibrium of percentage,
# so this is a very important variable to optimize as you search
# 0.5 represents 50% of the profit, 1 represents 100% of the profit
AUCTION_BID_PROFIT_PERCENTAGE = float(os.environ.get("AUCTION_BID_PROFIT_PERCENTAGE"))

FACTORY_CONTRACTS = json.loads(os.environ.get("FACTORY_CONTRACTS"))

cfg = NetworkConfig(
    chain_id=CHAIN_ID,
    url=f"rest+{REST_URL}",
    fee_minimum_gas_price=GAS_PRICE,
    fee_denomination=FEE_DENOM,
    staking_denomination=FEE_DENOM,
)
client = LedgerClient(cfg)

from terra_sdk.client.lcd import LCDClient
from terra_sdk.key.mnemonic import MnemonicKey

mk = MnemonicKey(mnemonic=MNEMONIC)
terra = LCDClient("https://phoenix-lcd.terra.dev", "phoenix-1")
terra_wallet = terra.wallet(mk)

wallet = LocalWallet(PrivateKey(terra_wallet.key.private_key), prefix=ADDRESS_PREFIX)

#j = await terraswap_factory(RPC_URL, "terra1466nf3zuxpya8q9emxukd7vftaf6h4psr0a07srl5zw74zh84yjqxl5qul")


In [2]:
FACTORY_CONTRACTS

{'factories': {'terraswap': 'terra1466nf3zuxpya8q9emxukd7vftaf6h4psr0a07srl5zw74zh84yjqxl5qul',
  'phoenix': 'terra1pewdsxywmwurekjwrgvjvxvv0dv2pf8xtdl9ykfce2z0q3gf0k3qr8nezy',
  'astroport': 'terra14x9fr055x5hvr48hzy2t4q7kvjvfttsvxusa4xsdcy702mnzsvuqprer8r',
  'white_whale': 'terra1f4cr4sr5eulp3f2us8unu6qv8a5rhjltqsg7ujjx6f2mrlqh923sljwhn3'}}

In [2]:
from base64 import b16decode, b16encode, b64decode, b64encode

def create_payload(contract_address: str, query: dict, height: str) -> dict:
    """Creates the payload for an abci_query"""
    data = QuerySmartContractStateRequest.SerializeToString(
                QuerySmartContractStateRequest(address=contract_address, 
                     query_data=json.dumps(query).encode('utf-8')))
    params = {"path": "/cosmwasm.wasm.v1.Query/SmartContractState",
              "data": b16encode(data).decode("utf-8"), "prove": False}
    if height:
        params["height"] = height

    print(params)
    
    payload = {"jsonrpc": "2.0",
               "id": 1,
               "method": "abci_query",
               "params": params}
    return payload

In [3]:
data = QuerySmartContractStateRequest.SerializeToString(
            QuerySmartContractStateRequest(address=contract_address, 
                    query_data=json.dumps(query).encode('utf-8')))
                    
params = {"path": "/cosmwasm.wasm.v1.Query/SmartContractState",
            "data": b16encode(data).decode("utf-8"), "prove": False}

NameError: name 'contract_address' is not defined

In [4]:
async def query_node_and_decode_response(rpc_url: str, payload: dict) -> dict:
    """Query node and decode response"""
    async with httpx.AsyncClient() as client:
        response = await client.post(rpc_url, json=payload)
        print(response)
        print(response.json())
    value = b64decode(response.json()["result"]["response"]["value"])
    decoded_value = json.loads(QuerySmartContractStateResponse.FromString(value).data.decode())
    return decoded_value

In [16]:
query = {"pool":{}}
payload = create_payload("terra1zrajvdc5yx0fsp429j6ej4nvrq68jjv078n94r00nl67d8cj6kmsalxtwt", query, height="")
await query_node_and_decode_response("https://terra-rpc.polkachu.com/", payload)

{'path': '/cosmwasm.wasm.v1.Query/SmartContractState', 'data': '0A407465727261317A72616A766463357978306673703432396A36656A346E76727136386A6A763037386E39347230306E6C36376438636A366B6D73616C78747774120C7B22706F6F6C223A207B7D7D', 'prove': False}
<Response [200 OK]>
{'jsonrpc': '2.0', 'id': 1, 'result': {'response': {'code': 0, 'log': '', 'info': '', 'index': '0', 'key': None, 'value': 'CswBeyJhc3NldHMiOlt7ImluZm8iOnsibmF0aXZlX3Rva2VuIjp7ImRlbm9tIjoidWx1bmEifX0sImFtb3VudCI6IjAifSx7ImluZm8iOnsidG9rZW4iOnsiY29udHJhY3RfYWRkciI6InRlcnJhMXZ6ZDk4czlrcWRrYXRhaHhzN3JzZDhtNDc0bGYyZjhjdDM5emRnZDZzaGo0bmg1ZTZrdXNrYXoyZ3kifX0sImFtb3VudCI6IjAifV0sInRvdGFsX3NoYXJlIjoiMCJ9', 'proofOps': None, 'height': '2983956', 'codespace': ''}}}


{'assets': [{'info': {'native_token': {'denom': 'uluna'}}, 'amount': '0'},
  {'info': {'token': {'contract_addr': 'terra1vzd98s9kqdkatahxs7rsd8m474lf2f8ct39zdgd6shj4nh5e6kuskaz2gy'}},
   'amount': '0'}],
 'total_share': '0'}

In [ ]:
{'assets': [{'info': {'token': {'contract_addr': 'terra1qj5hs3e86qn4vm9dvtgtlkdp550r0rayk9wpay44mfw3gn3tr8nq5jw3dg'}},
   'amount': '394615095747218'},
  {'info': {'native_token': {'denom': 'uluna'}}, 'amount': '4022069'}],
 'total_share': '37221339895'}

In [2]:
from update_contracts import update_all_factory_pools
contracts = {}
await update_all_factory_pools(contracts, RPC_URL, factory_contracts=FACTORY_CONTRACTS)

Total Pairs: 52
Updating pool info for terra1eud3zfx5q7eklahp3q9ar77gc432g7qa6lwm7z5zfg88qf80rvsshj29sg
Updating pool info for terra1eud3zfx5q7eklahp3q9ar77gc432g7qa6lwm7z5zfg88qf80rvsshj29sg terraswap
Updating pool info for terra1xjv2pmf26yaz3wqft7caafgckdg4eflzsw56aqhdcjw58qx0v2mqux87t8
Updating pool info for terra1xjv2pmf26yaz3wqft7caafgckdg4eflzsw56aqhdcjw58qx0v2mqux87t8 terraswap
Updating pool info for terra1wm9dlwgtufjnjzuuee8ftqy3t9vq728vhyxv0tuqgzk7dt3fmwwsecqh8j
Updating pool info for terra1wm9dlwgtufjnjzuuee8ftqy3t9vq728vhyxv0tuqgzk7dt3fmwwsecqh8j terraswap
Updating pool info for terra1qg85dekl59jv723ce54s82v26rteknru5645lfm3n9eytr53570ssrz6js
Updating pool info for terra1qg85dekl59jv723ce54s82v26rteknru5645lfm3n9eytr53570ssrz6js terraswap
Updating pool info for terra1gu75wek7kq8h4ee6eztmfu73nr3esl6al0qjawkhya3g57sz6jvsukpj3z
Updating pool info for terra1gu75wek7kq8h4ee6eztmfu73nr3esl6al0qjawkhya3g57sz6jvsukpj3z terraswap
Updating pool info for terra1j08452mqwadp8xu25kn9rleyl

In [2]:
with open('terra_contracts.json') as f:
    contracts = json.load(f)

In [5]:
contracts

{'terra1eud3zfx5q7eklahp3q9ar77gc432g7qa6lwm7z5zfg88qf80rvsshj29sg': {'info': {'parser': 'terraswap',
   'token1_type': 'token',
   'token1_denom': 'terra1q8kfp0v9rhef0d3u44ds9shwvwcusjheh8nhye3n7gwjd95ze96sehyp6w',
   'token2_type': 'native_token',
   'token2_denom': 'uluna'},
  'dex': 'terraswap',
  'routes': []},
 'terra1xjv2pmf26yaz3wqft7caafgckdg4eflzsw56aqhdcjw58qx0v2mqux87t8': {'info': {'parser': 'terraswap',
   'token1_type': 'token',
   'token1_denom': 'terra1qj5hs3e86qn4vm9dvtgtlkdp550r0rayk9wpay44mfw3gn3tr8nq5jw3dg',
   'token2_type': 'native_token',
   'token2_denom': 'uluna'},
  'dex': 'terraswap',
  'routes': [['terra1xjv2pmf26yaz3wqft7caafgckdg4eflzsw56aqhdcjw58qx0v2mqux87t8',
    'terra1wm9dlwgtufjnjzuuee8ftqy3t9vq728vhyxv0tuqgzk7dt3fmwwsecqh8j',
    'terra1w8246pdk9tf9d2dnu4lty5m8v3ptjltrm46vh8kd6yhr8k4ad2yskdqs6x'],
   ['terra1xjv2pmf26yaz3wqft7caafgckdg4eflzsw56aqhdcjw58qx0v2mqux87t8',
    'terra1wm9dlwgtufjnjzuuee8ftqy3t9vq728vhyxv0tuqgzk7dt3fmwwsecqh8j',
    'terra

In [4]:
generate_three_pool_cyclic_routes(contracts=contracts, arb_denom="uluna")

In [7]:
with open('terra_contracts.json', 'w') as f:
    json.dump(contracts, f, indent=4)

In [9]:
if "terra1xjv2pmf26yaz3wqft7caafgckdg4eflzsw56aqhdcjw58qx0v2mqux87t8" in contracts:
    print("hi")

hi


In [5]:
x = "eyJleGVjdXRlX3N3YXBfb3BlcmF0aW9ucyI6eyJvcGVyYXRpb25zIjpbeyJ0ZXJyYV9zd2FwIjp7Im9mZmVyX2Fzc2V0X2luZm8iOnsidG9rZW4iOnsiY29udHJhY3RfYWRkciI6InRlcnJhMTR4c20yd3p2dTd4YWY1NjdyNjkzdmdma2htdmZzMDhsNjhoNHRqajV3amd5bjVreThlMnF2enlhbmgifX0sImFza19hc3NldF9pbmZvIjp7Im5hdGl2ZV90b2tlbiI6eyJkZW5vbSI6InVsdW5hIn19fX0seyJ0ZXJyYV9zd2FwIjp7Im9mZmVyX2Fzc2V0X2luZm8iOnsibmF0aXZlX3Rva2VuIjp7ImRlbm9tIjoidWx1bmEifX0sImFza19hc3NldF9pbmZvIjp7InRva2VuIjp7ImNvbnRyYWN0X2FkZHIiOiJ0ZXJyYTFlZTRnNjNjM3N1czlobnl5cDNwMnUzdHVsemR2NWFnNjhsNTVxOGVqNjR5NHFwd3N3dnVzNW10YWcyIn19fX0seyJ0ZXJyYV9zd2FwIjp7Im9mZmVyX2Fzc2V0X2luZm8iOnsidG9rZW4iOnsiY29udHJhY3RfYWRkciI6InRlcnJhMWVlNGc2M2Mzc3VzOWhueXlwM3AydTN0dWx6ZHY1YWc2OGw1NXE4ZWo2NHk0cXB3c3d2dXM1bXRhZzIifX0sImFza19hc3NldF9pbmZvIjp7Im5hdGl2ZV90b2tlbiI6eyJkZW5vbSI6ImliYy9CMzUwNEUwOTI0NTZCQTYxOENDMjhBQzY3MUE3MUZCMDhDNkNBMEZEMEJFN0M4QTVCNUEzRTJERDkzM0NDOUU0In19fX1dLCJtaW5pbXVtX3JlY2VpdmUiOiIxMzgwIn19"
y = "eyJleGVjdXRlX3N3YXBfb3BlcmF0aW9ucyI6eyJvcGVyYXRpb25zIjpbeyJ0ZXJyYV9zd2FwIjp7Im9mZmVyX2Fzc2V0X2luZm8iOnsidG9rZW4iOnsiY29udHJhY3RfYWRkciI6InRlcnJhMTR4c20yd3p2dTd4YWY1NjdyNjkzdmdma2htdmZzMDhsNjhoNHRqajV3amd5bjVreThlMnF2enlhbmgifX0sImFza19hc3NldF9pbmZvIjp7Im5hdGl2ZV90b2tlbiI6eyJkZW5vbSI6InVsdW5hIn19fX0seyJ0ZXJyYV9zd2FwIjp7Im9mZmVyX2Fzc2V0X2luZm8iOnsibmF0aXZlX3Rva2VuIjp7ImRlbm9tIjoidWx1bmEifX0sImFza19hc3NldF9pbmZvIjp7InRva2VuIjp7ImNvbnRyYWN0X2FkZHIiOiJ0ZXJyYTFlZTRnNjNjM3N1czlobnl5cDNwMnUzdHVsemR2NWFnNjhsNTVxOGVqNjR5NHFwd3N3dnVzNW10YWcyIn19fX0seyJ0ZXJyYV9zd2FwIjp7Im9mZmVyX2Fzc2V0X2luZm8iOnsidG9rZW4iOnsiY29udHJhY3RfYWRkciI6InRlcnJhMWVlNGc2M2Mzc3VzOWhueXlwM3AydTN0dWx6ZHY1YWc2OGw1NXE4ZWo2NHk0cXB3c3d2dXM1bXRhZzIifX0sImFza19hc3NldF9pbmZvIjp7Im5hdGl2ZV90b2tlbiI6eyJkZW5vbSI6ImliYy9DQkY2N0EyQkNGNkNBRTM0M0ZERjI1MUU1MTBDOEUxOEMzNjFGQzAyQjIzNDMwQzEyMTExNkUwODExODM1REVGIn19fX1dLCJtaW5pbXVtX3JlY2VpdmUiOiI4NDkifX0="
z = "eyJzd2FwIjp7Im1heF9zcHJlYWQiOiIwLjAwNSIsImJlbGllZl9wcmljZSI6IjAuOTIwNjAzMDE1MDc1Mzc2ODg0In19"

In [6]:
import base64

x = base64.b64decode(x)
y = base64.b64decode(y)
z = base64.b64decode(z)

In [5]:
from query_contracts import terraswap_factory

In [6]:
j = await terraswap_factory(RPC_URL, "terra1466nf3zuxpya8q9emxukd7vftaf6h4psr0a07srl5zw74zh84yjqxl5qul")

ReadTimeout: 

In [10]:
from cosmpy.aerial.contract import LedgerContract

In [11]:
factory_contract = LedgerContract(path=None, client=client, address="terra1466nf3zuxpya8q9emxukd7vftaf6h4psr0a07srl5zw74zh84yjqxl5qul")

In [25]:
pairs_2 = factory_contract.query({"pairs": {"start_after": [{'native_token': {'denom': 'uluna'}},
  {'native_token': {'denom': 'ibc/CBF67A2BCF6CAE343FDF251E510C8E18C361FC02B23430C121116E0811835DEF'}}], "limit": 30}})

In [26]:
len(pairs_2["pairs"])

22

In [24]:
pairs["pairs"][-2]

{'asset_infos': [{'native_token': {'denom': 'uluna'}},
  {'native_token': {'denom': 'ibc/CBF67A2BCF6CAE343FDF251E510C8E18C361FC02B23430C121116E0811835DEF'}}],
 'contract_addr': 'terra1u3wd9gu7weezw6vwfaaa4q589zjlazg6wt6gyer3lc42tgqrpggqv90c2c',
 'liquidity_token': 'terra1w2l4w5p66l5t2nmrmsvz7k4cu50s7e8dc6h59gcxsnmp2tgy7q7smfaxql',
 'asset_decimals': [6, 6]}

In [27]:
pairs_2

{'pairs': [{'asset_infos': [{'token': {'contract_addr': 'terra1ee4g63c3sus9hnyyp3p2u3tulzdv5ag68l55q8ej64y4qpwswvus5mtag2'}},
    {'native_token': {'denom': 'ibc/CBF67A2BCF6CAE343FDF251E510C8E18C361FC02B23430C121116E0811835DEF'}}],
   'contract_addr': 'terra160lewlf0ygzvjkjar5n8wxulnh8phsu6vsq4sk8e3ln3pqz58juq22ywwy',
   'liquidity_token': 'terra1vz29w25qu5lzfghz89yy6cq7jaj5snjf5p66qcmp4hza87jcstfqylf5er',
   'asset_decimals': [6, 6]},
  {'asset_infos': [{'token': {'contract_addr': 'terra1dtaqwlmzlk3jku5un6h6rfunttmwsqnfz7evvdf4pwr0wypsl68q6nuam0'}},
    {'native_token': {'denom': 'uluna'}}],
   'contract_addr': 'terra1dtaakf99dllanxn0sg0ryft4j9fsewypgns5gavev6tz49mw0wds8cg89y',
   'liquidity_token': 'terra1q3647qp780u7y2zvau5fn748zqxsfm4kr6lcvr5jjev5a77kchxsjy2m4x',
   'asset_decimals': [6, 6]},
  {'asset_infos': [{'token': {'contract_addr': 'terra1d4j9lsl453mkvtlg4ctw8y52rdkhafsaefug0hq0z06phczuvvvs7uq0vg'}},
    {'native_token': {'denom': 'uluna'}}],
   'contract_addr': 'terra13ce38

In [4]:
from query_contracts import terraswap_info

In [31]:
o = await terraswap_info(RPC_URL, "terra1466nf3zuxpya8q9emxukd7vftaf6h4psr0a07srl5zw74zh84yjqxl5qul")

TypeError: argument should be a bytes-like object or ASCII string, not 'NoneType'